In [ ]:
# Librairie
import xarray as xr
import pandas as pd
import numpy as np

In [ ]:
# CONFIGURATION
path_data = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/data/raw-yearly-combined/era5_fr_t2m/*.nc"
path_mask = "C:/Users/margo/OneDrive/Documents/M1 SSD/Projet_Tutore/Analyse_Meteo/src/mask_france_boolean.nc"

In [ ]:
# TRAITEMENT
try:
    # On charge avec Dask (chunks) pour ne pas faire exploser la RAM
    ds = xr.open_mfdataset(path_data, chunks={'time': 500}, parallel=True)
    mask = xr.open_dataset(path_mask)

    # On applique le masque France (garde uniquement ce qui est dans le masque)
    ds_france = ds.where(mask['mask'] == 1)

    # --- FICHIER 1 : L'HISTORIQUE (CSV) ---
    print("📊 Calcul de la moyenne spatiale France (1950-2025)...")
    # Moyenne sur latitude/longitude -> On obtient une série temporelle
    ds_mean = ds_france.mean(dim=['latitude', 'longitude'], skipna=True)

    # Conversion en DataFrame Pandas
    df = ds_mean.to_dataframe().reset_index()

    # Nettoyage : On garde juste Date et Température (en °C)
    if 't2m' in df.columns:
        df['Temperature_C'] = df['t2m'] - 273.15
        final_df = df[['time', 'Temperature_C']].sort_values('time')

        # Sauvegarde en CSV (très léger, quelques Mo)
        final_df.to_csv("donnees_meteo_france_clean.csv", index=False)
        print("✅ CSV créé : 'donnees_meteo_france_clean.csv'")

    # --- FICHIER 2 : LA CARTE (NetCDF réduit) ---
    print("🗺️ Extraction de la carte pour la canicule 2003...")
    # On prend juste UN mois (Août 2003) pour faire la carte dans le TP
    ds_carte = ds_france.sel(time="2003-08-01", method="nearest")

    # Sauvegarde en petit NetCDF (quelques Ko)
    ds_carte.to_netcdf("carte_canicule_2003.nc")
    print("✅ NetCDF carte créé : 'carte_canicule_2003.nc'")

except Exception as e:
    print(f"❌ Erreur : {e}")
